In [70]:
import pandas as pd 
from numpy import array 
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

In [71]:
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [72]:
n_steps = 3

In [73]:
price_data = pd.read_csv('crop_dataset.csv')['Average']
print(price_data.head())

0    29.0
1    22.5
2    24.0
3    26.0
4    24.0
Name: Average, dtype: float64


In [74]:
weather_data = pd.read_csv('weather_dataset.csv')['tavg']
print(weather_data.head())

0    22.9
1    21.7
2    21.0
3    20.8
4    20.4
Name: tavg, dtype: float64


In [75]:
if len(weather_data) > len(price_data):
    weather_data = weather_data[:len(price_data)]

print(len(weather_data), len(price_data))

3077 3077


In [76]:
weather_data = array(weather_data)
price_data = array(price_data)

In [77]:
# convert to [rows, columns] structure
weather_data = weather_data.reshape((len(weather_data), 1))
price_data = price_data.reshape((len(price_data), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
print(out_seq)

[[51.9]
 [44.2]
 [45. ]
 ...
 [49.8]
 [56.6]
 [52.4]]


In [78]:
# horizontally stack columns
dataset = hstack((weather_data, price_data, out_seq))
print(dataset)

[[22.9 29.  51.9]
 [21.7 22.5 44.2]
 [21.  24.  45. ]
 ...
 [25.8 24.  49.8]
 [27.6 29.  56.6]
 [27.4 25.  52.4]]


In [79]:
X, y = split_sequence(dataset, n_steps)
print(X.shape, y.shape)

(3074, 3, 3) (3074, 3)


In [80]:
for i in range(len(X)):
 print(X[i], y[i])

[[22.9 29.  51.9]
 [21.7 22.5 44.2]
 [21.  24.  45. ]] [20.8 26.  46.8]
[[21.7 22.5 44.2]
 [21.  24.  45. ]
 [20.8 26.  46.8]] [20.4 24.  44.4]
[[21.  24.  45. ]
 [20.8 26.  46.8]
 [20.4 24.  44.4]] [20.4 17.5 37.9]
[[20.8 26.  46.8]
 [20.4 24.  44.4]
 [20.4 17.5 37.9]] [18.8 27.5 46.3]
[[20.4 24.  44.4]
 [20.4 17.5 37.9]
 [18.8 27.5 46.3]] [20.  22.5 42.5]
[[20.4 17.5 37.9]
 [18.8 27.5 46.3]
 [20.  22.5 42.5]] [21. 21. 42.]
[[18.8 27.5 46.3]
 [20.  22.5 42.5]
 [21.  21.  42. ]] [21.2 26.  47.2]
[[20.  22.5 42.5]
 [21.  21.  42. ]
 [21.2 26.  47.2]] [21.8 20.  41.8]
[[21.  21.  42. ]
 [21.2 26.  47.2]
 [21.8 20.  41.8]] [21.1 17.5 38.6]
[[21.2 26.  47.2]
 [21.8 20.  41.8]
 [21.1 17.5 38.6]] [18.8 17.5 36.3]
[[21.8 20.  41.8]
 [21.1 17.5 38.6]
 [18.8 17.5 36.3]] [19.8 16.  35.8]
[[21.1 17.5 38.6]
 [18.8 17.5 36.3]
 [19.8 16.  35.8]] [21. 16. 37.]
[[18.8 17.5 36.3]
 [19.8 16.  35.8]
 [21.  16.  37. ]] [22.1 21.  43.1]
[[19.8 16.  35.8]
 [21.  16.  37. ]
 [22.1 21.  43.1]] [23.8 22.5 46.3

In [82]:
n_features = X.shape[2]

In [83]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [84]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [92]:
x_input = array([[30, 24], [29, 24.5], [31, 24]])
print(x_input.shape)
x_input = x_input.reshape((1, n_steps, 2))
yhat = model.predict(x_input, verbose=0)

(3, 2)


ValueError: in user code:

    File "C:\Users\Hewlett Packard\major-project\venv\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Hewlett Packard\major-project\venv\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Hewlett Packard\major-project\venv\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Hewlett Packard\major-project\venv\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\Users\Hewlett Packard\major-project\venv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Hewlett Packard\major-project\venv